In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import sys
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
floating = pd.read_csv('data/skt_floating_pop.csv')

floating = floating.drop('datetime', axis=1).groupby(['gu', 'hour', 'age', 'sex']).sum().reset_index()
pop_by_gu = floating.drop('age', axis=1).groupby(['gu', 'hour']).sum('floating_pop').reset_index()

def get_percentage(gu, hour, floating_pop):
    total = pop_by_gu[(pop_by_gu.gu==gu)&(pop_by_gu.hour==hour)].floating_pop.values[0]
    return floating_pop/total

floating['percentage'] = floating.apply(lambda x: get_percentage(x.gu, x.hour, x.floating_pop), axis=1)
floating

In [ ]:
import re

boarding = pd.read_csv('data/seoul_boarding_pop_by_subway.csv')
def repl(name):
    if "승차" in name:
        return str(int(name.split("-")[0].replace("시", "")))+"_승차"
    else:
        return str(int(name.split("-")[0].replace("시", "")))+"_하차"
    
boarding = boarding.rename(columns={col: repl(col) for col in boarding.columns[3:-3]})
boarding['total'] = 0
for i in range(0, 24):
    boarding[i] = boarding[str(i)+"_승차"]+boarding[str(i)+"_하차"]
    boarding = boarding.drop([str(i)+"_승차", str(i)+"_하차"], axis=1)
    boarding['total'] += boarding[i]
boarding

In [ ]:
ages = floating.age.unique()
sexes = floating.sex.unique()
age_sex = [(age, sex) for age in ages for sex in sexes]

times = np.arange(0, 24)
df = pd.DataFrame(columns=["id", "line", "station", "gu", "hour" "age", "sex", "estimated_pop"])

for i, row in boarding.iterrows():
    print(len(df))
    _id, line, station, gu = row.ID, row.line, row.station, row.gu
    for hour in times:
        total_pop = row[hour]
        floating_pop = floating[(floating.gu==gu)&(floating.hour==hour)]
        for j, _row in floating_pop.iterrows():
            age, sex, percentage = _row.age, _row.sex, _row.percentage
            data = total_pop * percentage
            df = df.append([_id, line, station, gu, hour, age, sex, data])
df.to_csv('data/estimated_pop.csv')